# Data Wrangling

The Apple (AAPL) stock data is from the CRSP database. The sample period spans 6 years from 2013 to 2018. 

The data wrangling process addresses the following:
1. Data type
2. Missing values
3. Variable adjustments
4. Feature creation
5. Drop unnecessary columns

First, import the AAPL stock data as a dataframe and inspect the data. Check the data type and look for missing values.

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# import AAPL stock data
AAPL = pd.read_csv('/Users/meiliu/Documents/SpringBoard Related/AAPL_Stock.csv')
AAPL.head()

,PERMNO,date,TICKER,DCLRDT,DIVAMT,BIDLO,ASKHI,PRC,VOL,BID,ASK,SHROUT,CFACPR,CFACSHR,OPENPRC,NUMTRD,sprtrn
0,14593,01/02/2013,AAPL,NaN,NaN,541.63000,555.00000,549.03003,21037195,548.70001,548.71002,938973,7,7,553.82001,151041,0.025403
1,14593,01/03/2013,AAPL,NaN,NaN,541.00000,549.66998,542.09589,13297102,542.27002,542.44000,938973,7,7,547.88000,95452,-0.002086
2,14593,01/04/2013,AAPL,NaN,NaN,525.82861,538.62988,527.00000,22087524,526.81000,527.01001,938973,7,7,536.96503,151801,0.004865
3,14593,01/07/2013,AAPL,NaN,NaN,515.20001,529.29999,523.90002,18017331,523.96997,524.19000,938973,7,7,522.00000,127922,-0.003123
4,14593,01/08/2013,AAPL,NaN,NaN,521.25000,531.89001,525.31000,16916775,525.15997,525.40002,938973,7,7,529.21002,113763,-0.003242


In [3]:
# check the AAPL dataframe
print(AAPL.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510 entries, 0 to 1509
Data columns (total 17 columns):
PERMNO     1510 non-null int64
date       1510 non-null object
TICKER     1510 non-null object
DCLRDT     25 non-null object
DIVAMT     25 non-null float64
BIDLO      1510 non-null float64
ASKHI      1510 non-null float64
PRC        1510 non-null float64
VOL        1510 non-null int64
BID        1510 non-null float64
ASK        1510 non-null float64
SHROUT     1510 non-null int64
CFACPR     1510 non-null int64
CFACSHR    1510 non-null int64
OPENPRC    1510 non-null float64
NUMTRD     1510 non-null int64
sprtrn     1510 non-null float64
dtypes: float64(8), int64(6), object(3)
memory usage: 200.7+ KB
None


There are 1510 tradings days over the 6 year period. There are three variables of type object. The only variables with missing values are DCLRDT and DIVAMT.

### 1. Data type

The three variables with type object are: date, TICKER, and DCLRDT. Convert the date to a datetime format. The ticker is not useful in the analysis and will be dropped later. Convert DCLRDT to int64 after making a dummy for the declaration date.

In [4]:
# convert date to DateTime format 
AAPL['date'] = pd.to_datetime(AAPL['date'])

### 2. Missing values

In [5]:
# check the non NaN values for DCLRDT and DIVAMT (declaration date and dividend amount)
AAPL[AAPL['DCLRDT'].notnull()][['DCLRDT', 'DIVAMT']].head()

,DCLRDT,DIVAMT
25,01/23/2013,2.65
88,04/23/2013,3.05
151,07/23/2013,3.05
214,10/28/2013,3.05
276,01/27/2014,3.05


Dividends are only paid on the dividend distribution date, so the nan values can be set to 0. Since DCLRDT is the announcement date, it makes sense to make it a dummy variable where DCLRDT=1 on the dividend declaration date and =0 otherwise. Make sure the variable type is converted from object to int.

In [6]:
# fill NaN values in DIVAMT as 0
AAPL['DIVAMT'].fillna(0, inplace=True)

# set DCLRDT on the decclaration dates =1 and =0 otherwise
dlr_dates = list(AAPL['DCLRDT'][AAPL['DCLRDT'].notnull()])

AAPL['DCLRDT'][AAPL['date'].isin(dlr_dates)] = 1
AAPL['DCLRDT'][~AAPL['date'].isin(dlr_dates)] = 0

# convert the type to int64
AAPL['DCLRDT'] = AAPL['DCLRDT'].astype('int64')

### 3. Variable adjustments

Adjust the variables related to price by the the cumulative adjustment factor for price (CFACPR) and the variables related to the number of shares by the cumulative adjustment factor for shares (CFASHR). This adjusts the variables for the 7-1 stock split on 06/09/2014 so that the values will not suddenly jump on that date.

In [7]:
# adjust variables related to price by CFACPR 
AAPL['PRC'] = AAPL['PRC']/AAPL['CFACPR']
AAPL['OPENPRC'] = AAPL['OPENPRC']/AAPL['CFACPR']
AAPL['BIDLO'] = AAPL['BIDLO']/AAPL['CFACPR']
AAPL['ASKHI'] = AAPL['ASKHI']/AAPL['CFACPR']
AAPL['BID'] = AAPL['BID']/AAPL['CFACPR']
AAPL['ASK'] = AAPL['ASK']/AAPL['CFACPR']
AAPL['DIVAMT'] = AAPL['DIVAMT']/AAPL['CFACPR']

# adjust the variables related to number of shares by the CFASHR
AAPL['SHROUT'] = AAPL['SHROUT']*AAPL['CFACSHR']
AAPL['VOL'] = AAPL['VOL']*AAPL['CFACSHR']
AAPL['NUMTRD'] = AAPL['NUMTRD']*AAPL['CFACSHR']

### 4. Feature creation

Create new features for the dayofweek, month, and year to use for EDA. Add percentage bid-ask spreads and the percentage difference between opening and closing prices as additional features.

In [8]:
# add dayofweek, month, year features
# dayofweek: where Mon=0 and Fri=6
AAPL['dayofweek'], AAPL['month'], AAPL['year'] = AAPL.date.dt.dayofweek, AAPL.date.dt.month, AAPL.date.dt.year

# calculate the bid ask spreads
AAPL['HISPRD'] = (AAPL['ASKHI'] - AAPL['BIDLO'])/AAPL['BIDLO']
AAPL['SPRD'] = (AAPL['ASK'] - AAPL['BID'])/AAPL['BID']

# calculate the diff between opening and closing prices
AAPL['OCdiff'] = (AAPL['OPENPRC'] - AAPL['PRC'])/AAPL['PRC']

#### <i>Google Trends

Google Trends data is only available on a monthly basis for my sample period. To add it as a predictor, merge the previous month's trend data to each date in the AAPL data. 

In [9]:
from pytrends.request import TrendReq

# initialize trend request
pytrends = TrendReq(hl='en-US', tz=360)

# set keyword(s)
kw_list = ['AAPL share price']

# request data over sample period -1 month
pytrends.build_payload(kw_list, cat=0, timeframe='2012-12-01 2018-12-31', geo='', gprop='')

# convert interest over time to a dataframe
df = pytrends.interest_over_time()
df.rename(columns={kw_list[0]:'TREND'}, inplace=True)

# offset the google trends data by +1 month
# add year and month in order to merge to AAPL
df.reset_index(inplace=True)
df['date'] = pd.to_datetime(df['date']) + pd.DateOffset(months=1)
df['year'] = df.date.dt.year
df['month'] = df.date.dt.month
df.head()

,date,TREND,isPartial,year,month
0,2013-01-01,19,False,2013,1
1,2013-02-01,43,False,2013,2
2,2013-03-01,28,False,2013,3
3,2013-04-01,26,False,2013,4
4,2013-05-01,27,False,2013,5


In [11]:
# merge the google trends and the AAPL data by year and month
AAPL_merge = pd.merge(AAPL, df[['year', 'month','TREND']], on=['year', 'month'], how='left')

#### <i>Earnings Announcement Data

The data is parsed from: https://www.streetinsider.com/ec_earnings.php?sort=earning_date&q=AAPL

Use the earnings announcement information to try and capture some of the effects of post earnings announcement drift. To do so, create a variable that counts the number of days from the earnings announcement date. This Announcement variable =30 on the announcement day and decays each trading day by 1. The variable identifies the 30 trading days on/after the earnings announcement. The earnings surprise helps determine the direction of the drift, so the percentage earnings surprise is forward filled from each announcement date.

In [12]:
# import earning announcement tsv
df = pd.read_csv("/Users/meiliu/Documents/SpringBoard Related/AAPL_earnings.tsv",sep='\t', header=None)
df.columns = ['date', 'QTR', 'EPS', 'EEPS', 'Surprise']

# convert date to datetime format
df['date'] = pd.to_datetime(df['date'])

# convert string values to float
df['EPS'] = df['EPS'].replace('[\$,]', '', regex=True).astype(float)
df['EEPS'] = df['EEPS'].replace('[\$,]', '', regex=True).astype(float)
df['Surprise'] = df['Surprise'].replace('[\$,]', '', regex=True).astype(float)

df.head() 

,date,QTR,EPS,EEPS,Surprise
0,2010-10-18,Q410,0.66,0.58,0.08
1,2011-01-18,Q111,0.92,0.77,0.15
2,2011-04-20,Q211,0.91,0.76,0.15
3,2011-07-19,Q311,1.11,0.83,0.28
4,2011-10-18,Q411,1.01,1.04,-0.03


In [13]:
# merge the google trends and the AAPL data by year and month
AAPL_merge = pd.merge(AAPL_merge, df[['date','EPS', 'EEPS']], on='date', how='left')

# create an announcment variable for the 30 trading days on/after the earnings announcement
# (where announcement day = 30 and 29 trading days later = 1)
AAPL_merge['Announce'] =  np.where(AAPL_merge['EPS'].notna(), 30, 0)

for i in range(30):
    AAPL_merge['Announce'] = np.maximum(AAPL_merge['Announce'].values, 
                                        (AAPL_merge['Announce'].shift()-1).fillna(0).values)

# forward fill the earnings surprise
# ignore NaN values in Surprise since they will be dropped in the XGBoost model
AAPL_merge['Surprise'] = (AAPL_merge['EPS']-AAPL_merge['EEPS'])/AAPL_merge['EEPS']
AAPL_merge['Surprise'] = AAPL_merge['Surprise'].fillna(method='ffill')

### 5. Drop unnecessary columns

In [14]:
#print(AAPL_merge.columns)
droplist = ['PERMNO', 'TICKER', 'BIDLO', 'ASKHI', 'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'EPS', 'EEPS']
AAPL_clean = AAPL_merge.drop(droplist, axis=1)

AAPL_clean.head()

,date,DCLRDT,DIVAMT,PRC,VOL,OPENPRC,NUMTRD,sprtrn,dayofweek,month,year,HISPRD,SPRD,OCdiff,TREND,Announce,Surprise
0,2013-01-02,0,0.0,78.432861,147260365,79.117144,1057287,0.025403,2,1,2013,0.024685,0.000018,0.008724,19,0.0,NaN
1,2013-01-03,0,0.0,77.442270,93079714,78.268571,668164,-0.002086,3,1,2013,0.016026,0.000313,0.010670,19,0.0,NaN
2,2013-01-04,0,0.0,75.285714,154612668,76.709290,1062607,0.004865,4,1,2013,0.024345,0.000380,0.018909,19,0.0,NaN
3,2013-01-07,0,0.0,74.842860,126121317,74.571429,895454,-0.003123,0,1,2013,0.027368,0.000420,-0.003627,19,0.0,NaN
4,2013-01-08,0,0.0,75.044286,118417425,75.601431,796341,-0.003242,1,1,2013,0.020412,0.000457,0.007424,19,0.0,NaN


In [16]:
AAPL_clean.to_csv('/Users/meiliu/Documents/SpringBoard Related/AAPL_clean.csv')